In [ ]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
import numpy as np

In [ ]:
from gensim.models.word2vec import Text8Corpus

In [ ]:
from gensim.test.utils import datapath
from gensim.models.word2vec import LineSentence

In [ ]:
sentences = LineSentence(datapath('/content/data_train.txt'))
index = 0
for sentence in sentences:
  print(sentence)
  index += 1
  if index > 10:
    break

In [ ]:
len(sentences)

In [ ]:
from gensim.models.callbacks import CallbackAny2Vec
class EpochSaver(CallbackAny2Vec):
	def __init__(self,path_prefix):
		self.epoch=0
	def on_epoch_end(self,model):
		print("Epoch_"+str(self.epoch))
		print("Training loss:"+str(model.get_latest_training_loss()))
		self.epoch+=1
epoch_saver=EpochSaver("model_some")

In [ ]:
model = Word2Vec(sentences=sentences, min_count=2, size=300, window=1,iter=150, workers=4,compute_loss=True,callbacks=[epoch_saver])

In [ ]:
model.wv.most_similar(['secure_EN_verb'], topn=20)

In [ ]:
model.save('/content/drive/MyDrive/model/model_word2vec_150_iters')

In [ ]:
model = Word2Vec.load('/content/drive/MyDrive/model/model_word2vec_150_iters')

In [ ]:
model.wv.most_similar(['Artini_FR_properNoun'], topn=20)

[('timorenc_CA_adjective', 0.9254214763641357),
 ('Giovanardi_FR_properNoun', 0.9237130284309387),
 ('Itter_CA_properNoun', 0.921836793422699),
 ('Enciclopèdia_Catalana_FR_properNoun', 0.921768069267273),
 ('mi-janvier_FR_noun', 0.9214802980422974),
 ('los_Abruzos_ES_properNoun', 0.9213172197341919),
 ('Montes_Balcanes_ES_properNoun', 0.9209089875221252),
 ('arbitratif_FR_adjective', 0.9208766222000122),
 ('rowing_club_of_Hondarribia_EN_properNoun', 0.9203571081161499),
 ('triper_CA_noun', 0.9200819730758667),
 ('aeronaŭtike_EO_adverb', 0.9195341467857361),
 ('Lev_FR_properNoun', 0.9194321036338806),
 ('in_a_skyward_direction_EN_adverb', 0.9191176891326904),
 ('hermita_formo_EO_noun', 0.9188501834869385),
 ('Elvio_CA_properNoun', 0.9188141822814941),
 ('Ivașcu_CA_properNoun', 0.9187545776367188),
 ('parièr_OC_noun', 0.9186429977416992),
 ('Celas_ES_properNoun', 0.9185701608657837),
 ('sheller_EN_noun', 0.9181265234947205),
 ('hétérodiégétique_FR_adjective', 0.9176884889602661)]

In [ ]:
vocabularies = list(model.wv.vocab.keys())

In [ ]:
EN_vocab = [x for x in vocabularies if '_EN_' in x]
EN_vectors = model.wv[EN_vocab]
FR_vocal = [x for x in vocabularies if '_FR_' in x]
FR_vectors = model.wv[FR_vocal]

In [ ]:
print(len(FR_vocal))
len(EN_vocab)

82200


59577

In [ ]:
a = 'rowing_club_of_Hondarribia_EN_properNoun'
subs = a.split('_')
pos = subs[-1]
" ".join(subs[0:len(subs)-2])

'rowing club of Hondarribia'

In [ ]:
def get_word_pos(x):
  subs = x.split('_')
  word = " ".join(subs[0:len(subs)-2])
  pos = subs[-1]
  return word, pos

In [ ]:
target_vocab = FR_vocal
target_vectors = FR_vectors
def find_translate(in_word):
  source_vec = model.wv[in_word]
  word_source, pos_source = get_word_pos(in_word)
  cosine_cosine_simi = model.wv.cosine_similarities(source_vec, target_vectors)
  sorted_index = np.argsort(-cosine_cosine_simi)
  for index in sorted_index:
    word_target, pos_target = get_word_pos(target_vocab[index])
    if pos_source == pos_target:
      return word_source + '\t' + word_target + '\t' + pos_source + '\t' + str(np.round(cosine_cosine_simi[index],2))

In [ ]:
array = []
for en_word in EN_vocab:
  array.append(find_translate(en_word, FR_vocal, FR_vectors))

In [ ]:
from multiprocessing import Pool
p = Pool(5)
array = p.map(find_translate, EN_vocab)  

In [ ]:
with open('/content/drive/MyDrive/model/dict.tsv', 'w') as f:
  for item in array:
    f.write("%s\n" % item)

In [ ]:
len(array)

59577